<a href="https://colab.research.google.com/github/MihneaGherghel/ColabCopy/blob/main/stockpredictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
class OptainData:
  def __init__(self,file_path):
    self.data = pd.read_csv(file_path, encoding='unicode_escape')
    self.data['Label'] = pd.to_numeric(self.data['Label'], errors='coerce')
    self.data = self.data.dropna()
    self.data['Label'] = self.data['Label'].astype(int)

  def split_labels_samples(self, data):
    labels=np.array([int(x) for x in data.iloc[:, 1]])
    samples=np.array(data.iloc[:, 2:])
    return self.clean_sentance(samples),labels

  def clean_sentance(self,sentence):
    sentence=[[re.sub('[^a-zA-Z]',' ', x) for x in y] for y in sentence]
    sentence=[[x.lower for x in y] for y in sentence]
    return sentence;

  def train_test(self):
    train_df=self.data[self.data['Date']<'20120101']
    val_df=self.data[self.data['Date']>'20141231']
    return train_df,val_df

In [ ]:
data=OptainData('./gdrive/MyDrive/DataStock.csv')
train_df,test_df=data.train_test()
train_samples,train_labels=data.split_labels_samples(train_df)
test_samples,test_labels=data.split_labels_samples(test_df)


In [ ]:
text=[]
for row in range(0,len(train_samples)):
  text.append(' '.join(str(x) for x in train_samples[row][0:25]))

In [ ]:
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(text)

In [ ]:
randomclassifier=RandomForestClassifier(n_estimators=200, criterion='entropy')
randomclassifier.fit(traindataset,train_labels)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [ ]:
text_test=[]
for row in range(0,len(test_samples)):
  text_test.append(' '.join(str(x) for x in train_samples[row][0:25]))
data_test=countvector.transform(text_test)
predictions=randomclassifier.predict(data_test)
print(accuracy_score(predictions,test_labels))
print(confusion_matrix(predictions,test_labels))

0.5052910052910053
[[94 95]
 [92 97]]
